In [ ]:
from main import *
device = 'cpu'

100%|██████████| 986/986 [00:00<00:00, 7303.56it/s]


In [3]:
vocab_src, vocab_trg, train_dataset, val_dataset

(<dataset.Vocab at 0x283491cc830>,
 <dataset.TranslationDataset at 0x2836d6b7250>)

In [4]:
from dataset import TranslationDataset, Vocab, TrainDataLoader, TestDataLoader
from config import filenames, folders
from lstm3 import LSTM_3

In [5]:
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3

train_loader = TrainDataLoader(train_dataset, shuffle=True)
val_loader = TestDataLoader(val_dataset, shuffle=False)

weights_filename = folders['weights'] + 'LSTM_3-testing-tf-metric-30.0m-15epoch.pt'

model = LSTM_3(config=config, weights_filename=weights_filename).to(device)
print(model)

LSTM_3(
  (src_embedding): Embedding(21665, 120, padding_idx=1)
  (trg_embedding): Embedding(18710, 120, padding_idx=1)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (enc_dropout): Dropout(p=0.1, inplace=False)
  (dec_dropout): Dropout(p=0.1, inplace=False)
  (attention_dropout): Dropout(p=0.1, inplace=False)
  (encoder): LSTM(120, 360, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  (encoder_output_proj): Linear(in_features=720, out_features=360, bias=True)
  (decoder): LSTM(120, 360, num_layers=3, batch_first=True, dropout=0.1)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=720, out_features=360, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=720, out_features=360, bias=True)
  )
  (fc): Linear(in_features=720, out_features=18710, bias=True)
)


In [6]:
import numpy as np
np.load('../weights/train.npy')

array([4.2534399 , 3.76737356, 3.56576598, 3.47964174, 3.42476594,
       3.39220035, 3.40126699, 3.40977442, 3.38240802, 3.40251803,
       3.41449302, 3.42746729, 3.42991304, 3.43507791, 3.43585044])

In [7]:
from submission import get_bleu

In [8]:
from dataset import RawDataset
raw_dataset = RawDataset(filenames['test_src'])

In [9]:
model.to(device)
get_bleu(model, dataloader=val_loader, vocab_trg=vocab_trg, filenames=filenames, device=device, raw_dataset=raw_dataset)

100%|██████████| 4/4 [00:10<00:00,  2.66s/it]


27.18

In [375]:
import torch
val_loader = TestDataLoader(val_dataset, batch_size=1, shuffle=False)
for src_seq, _ in val_loader:
    batch_size = src_seq.size(1)
    trg_seq = torch.tensor([[bos_idx]] * batch_size, dtype=torch.long).to(device)  # (batch_size, 1)
    print(batch_size)
    break

20


In [15]:
model=model.to('cuda')

In [20]:
model.demonstrate(val_loader, vocab_src, vocab_trg, device='cuda', examples=5)

19 21 20
src:	 als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freude geweckt .
trg:	 when i was <NUM> , i remember waking up one morning to the sound of joy in my house .
pred:	 when i was <NUM> years old , i was by the sound of the sound of .
17 15 17
src:	 mein vater hörte sich auf seinem kleinen , grauen radio die der bbc an .
trg:	 my father was listening to bbc news on his small , gray radio .
pred:	 my father listened to his little , gray listening to the of the bbc .
20 21 23
src:	 er sah sehr glücklich aus , was damals ziemlich ungewöhnlich war , da ihn die nachrichten meistens .
trg:	 there was a big smile on his face which was unusual then , because the news mostly depressed him .
pred:	 he looked very happy , which was pretty unusual at the time , because the news was most of the time .
11 12 11
src:	 er rief : " die taliban sind weg ! "
trg:	 " the taliban are gone ! " my father .
pred:	 he said , " the taliban are gone ! "
20 22 21
src:	 ich w

KeyboardInterrupt: 

In [19]:
from time import sleep
sleep(0)

In [21]:
raw_submission = RawDataset(filenames['submission_src'])

In [22]:
from submission import make_submission
from dataset import SubmissionDataset, SubmissionDataLoader
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab_src, device=device)
ldr = SubmissionDataLoader(submission_dataset)
make_submission(model, ldr, vocab_trg, filenames, device=device, raw_dataset=raw_submission)

  0%|          | 0/24 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [15]:
src, trg = val_dataset[0]

In [210]:
from dataset import TestDataLoader
ldr = TestDataLoader(val_dataset, batch_size=2)
src, trg = ldr[0]
src.size(1)

20

In [215]:
predictions = model.inference(src, device='cpu')
predictions

tensor([[    2,  8125, 18184,     4, 18611,  8125, 18184, 18184, 11593, 10819,
         10819, 10819, 10819, 10819, 16813, 11529, 16813, 11529,    32,    32,
            32,    32,     3],
        [    2,  6209, 18184, 11587,  7839,    28,  9758,    28,    28,  1543,
            28,    28, 13407, 16813, 16813, 11529, 16813, 11529, 16813,    32,
             3,     3,     3]])

In [218]:
import torch
from torch.nn.functional import softmax
trg_words = ['<BOS>', 'i', 'was', '<NUM>', 'years', 'i', 'was']
i = len(trg_words) - 1
trg_pred = torch.tensor([vocab_trg.encode_word(word) for word in trg_words], dtype=torch.long).unsqueeze(0)
predictions = model.inference(src, device='cpu')
print(vocab_trg.decode(trg_pred.squeeze(0).numpy()))
for idx in predictions[0]:
    print(vocab_trg.decode_idx(idx.item()), end=' ')

['i', 'was', '<NUM>', 'years', 'i', 'was']
<BOS> i was <NUM> years i was was one morning morning morning morning morning the of the of . . . . <EOS> 

In [183]:
vocab_trg.decode(model.inference(src, device='cpu').squeeze(0))

['i',
 'was',
 '<NUM>',
 'years',
 'i',
 'was',
 'was',
 'one',
 'morning',
 'morning',
 'morning',
 'morning',
 'morning',
 'the',
 'of',
 'the',
 'of',
 '.',
 '.',
 '.',
 '.']

In [154]:
trg_pred.squeeze(0).numpy()

array([    2,  8125, 18184,     4, 18611,  8125, 18184])